In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os 

### THIS IS MEANT TO RUN ON NERO - NEEDS TO BE CHANGED IF YOU RUN LOCALLY
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/ccorbin/.config/gcloud/application_default_credentials.json' 
os.environ['GCLOUD_PROJECT'] = 'mining-clinical-decisions' 
%load_ext google.cloud.bigquery

from google.cloud import bigquery
client=bigquery.Client()

In [ ]:
%%bigquery
# Of our 60k encounters, how many didn't have any abx started
WITH start_stop_time as (
SELECT anon_id, pat_enc_csn_id_coded, max(was_given) any_abx_given, min(order_start_time_utc) first_abx_order_time, max(abx_stop_time) last_abx_stop_time
FROM `mining-clinical-decisions.conor_db.abx_temp_14_day_orders` 
GROUP BY anon_id, pat_enc_csn_id_coded
ORDER BY anon_id, pat_enc_csn_id_coded)

SELECT any_abx_given, COUNT (DISTINCT pat_enc_csn_id_coded) cnt
FROM start_stop_time 
GROUP BY any_abx_given 

In [ ]:
%%bigquery
# How many CSNS have abx that are stopped within given number of days. Shows distribution.  Weird how long the tail is, should investigate further. 
# Also note how 1/3 of the CSN's have abx stop within the first day. Note that these will include people sent home with PO abx who were probably
# taking them for the full week. May end up wanting to remove PO abx from this analysis bc we can't get a stop time for them. 
WITH start_stop_time as (
SELECT anon_id, pat_enc_csn_id_coded, max(was_given) any_abx_given, min(order_start_time_utc) first_abx_order_time, max(abx_stop_time) last_abx_stop_time
FROM `mining-clinical-decisions.conor_db.abx_temp_14_day_orders` 
GROUP BY anon_id, pat_enc_csn_id_coded
ORDER BY anon_id, pat_enc_csn_id_coded),

er_admits AS (
SELECT anon_id, pat_enc_csn_id_coded, min(effective_time_jittered_utc) as er_admit_time, max(effective_time_jittered_utc) as er_transfer_out_time
FROM `shc_core.adt`
WHERE pat_class_c = "112" AND pat_service = "Emergency"
GROUP BY anon_id, pat_enc_csn_id_coded)

SELECT
TIMESTAMP_DIFF(ss.last_abx_stop_time, ea.er_admit_time, DAY) time_until_abx_stop, COUNT (DISTINCT ss.pat_enc_csn_id_coded) cnt
FROM start_stop_time ss
INNER JOIN er_admits ea
USING (pat_enc_csn_id_coded)
GROUP BY time_until_abx_stop
ORDER BY time_until_abx_stop


In [ ]:
%%bigquery
# Counts the number of CSNS that that have any growth in the union of all cultures ordered within 4 hrs of er stay
WITH cult_result as (
SELECT DISTINCT co.anon_id, co.pat_enc_csn_id_coded, co.order_proc_id_coded, cs.organism
FROM `mining-clinical-decisions.conor_db.abx_culture_orders_within_4_hours` co
INNER JOIN 
  (SELECT DISTINCT pat_enc_csn_id_coded FROM `mining-clinical-decisions.conor_db.abx_med_orders_given_and_stopped_info`) cults_and_abx_csns
USING (pat_enc_csn_id_coded)
LEFT JOIN `shc_core.culture_sensitivity` cs
USING (order_proc_id_coded)
ORDER BY co.anon_id, co.pat_enc_csn_id_coded
)

SELECT any_growth, COUNT (DISTINCT pat_enc_csn_id_coded) num_csns
FROM
  (SELECT pat_enc_csn_id_coded, MAX(CASE WHEN organism IS NOT NULL THEN 1 ELSE 0 END) any_growth
  FROM cult_result
  GROUP BY pat_enc_csn_id_coded) t
GROUP BY any_growth


In [ ]:
%%bigquery
# Of all encs with positive cultures, how many different kinds of bugs grew?
WITH cult_result as (
SELECT DISTINCT co.anon_id, co.pat_enc_csn_id_coded, co.order_proc_id_coded, cs.organism
FROM `mining-clinical-decisions.conor_db.abx_culture_orders_within_4_hours` co
INNER JOIN 
  (SELECT DISTINCT pat_enc_csn_id_coded FROM `mining-clinical-decisions.conor_db.abx_med_orders_given_and_stopped_info`) cults_and_abx_csns
USING (pat_enc_csn_id_coded)
LEFT JOIN `shc_core.culture_sensitivity` cs
USING (order_proc_id_coded)
ORDER BY co.anon_id, co.pat_enc_csn_id_coded
),

any_growth as 
(SELECT pat_enc_csn_id_coded, MAX(CASE WHEN organism IS NOT NULL THEN 1 ELSE 0 END) any_positive
FROM cult_result
GROUP BY pat_enc_csn_id_coded)

SELECT num_bugs, COUNT (DISTINCT pat_enc_csn_id_coded) num_csns
FROM 
  (SELECT cr.pat_enc_csn_id_coded, COUNT (DISTINCT cr.organism) num_bugs
  FROM cult_result cr
  INNER JOIN (SELECT DISTINCT pat_enc_csn_id_coded FROM any_growth WHERE any_positive = 1) pos_csns
  USING (pat_enc_csn_id_coded)
  GROUP BY cr.pat_enc_csn_id_coded) t
GROUP BY num_bugs
ORDER BY num_bugs

In [ ]:
%%bigquery
# Kinds of bugs that grow and how often stratified by department (Pediatric ER, Adult ER, Valley Care ER).  Scary thing here is most of these are pediatric.  Which I was going to exclude
# from the cohort... Note the Pediatric and adult ER's are the same building I believe.  
WITH cult_result as (
SELECT DISTINCT co.anon_id, co.pat_enc_csn_id_coded, co.order_proc_id_coded, cs.organism
FROM `mining-clinical-decisions.conor_db.abx_culture_orders_within_4_hours` co
INNER JOIN 
  (SELECT DISTINCT pat_enc_csn_id_coded FROM `mining-clinical-decisions.conor_db.abx_med_orders_given_and_stopped_info`) cults_and_abx_csns
USING (pat_enc_csn_id_coded)
LEFT JOIN `shc_core.culture_sensitivity` cs
USING (order_proc_id_coded)
ORDER BY co.anon_id, co.pat_enc_csn_id_coded
),

any_growth as 
(SELECT pat_enc_csn_id_coded, MAX(CASE WHEN organism IS NOT NULL THEN 1 ELSE 0 END) any_positive
FROM cult_result
GROUP BY pat_enc_csn_id_coded), 

dep_id as (
SELECT 
adt.pat_enc_csn_id_coded,
FIRST_VALUE(adt.department_id) OVER (PARTITION BY adt.pat_enc_csn_id_coded ORDER BY adt.effective_time_jittered_utc) department_id
FROM cult_result
INNER JOIN `shc_core.adt` adt
USING (pat_enc_csn_id_coded)
)

SELECT department_name, organism, COUNT (DISTINCT pat_enc_csn_id_coded) num_csns
FROM 
  (SELECT DISTINCT cr.pat_enc_csn_id_coded, cr.organism, dm.department_name
  FROM cult_result cr
  INNER JOIN (SELECT DISTINCT pat_enc_csn_id_coded FROM any_growth WHERE any_positive = 1) pos_csns
  USING (pat_enc_csn_id_coded)
  INNER JOIN dep_id
  USING (pat_enc_csn_id_coded)
  INNER JOIN `shc_core.dep_map` dm
  USING (department_id)
  WHERE cr.organism IS NOT NULL) t
GROUP BY department_name, organism
ORDER BY department_name, num_csns DESC

In [ ]:
%%bigquery
# Counts of infected, not infected, and censored patients.  Roughly half of patients are censored according to first round logic.
SELECT sum(not_infected), sum(infected), sum(not_censored)
FROM `mining-clinical-decisions.conor_db.abx_cohort_not_censored_label`


In [ ]:
%%bigquery
# Counts of infected, not infected, and censored patients when kids removed -- still roughly half censored. more infected than uninfected
SELECT sum(not_infected), sum(infected), sum(not_censored)
FROM `mining-clinical-decisions.conor_db.abx_cohort_not_censored_no_kids`


In [ ]:
%%bigquery
# Counts of infected, not infected, and censored patients when kids removed -- still roughly half censored. more infected than uninfected
SELECT sum(not_infected), sum(infected), sum(not_censored)
FROM `mining-clinical-decisions.conor_db.abx_cohort_not_censored_no_kids_no_prior`


In [ ]:
%%bigquery
# get dep_names for the Ed for cohort without kids, lot of pediatric even though we're 18+.
WITH adt_dep as (
SELECT pat_enc_csn_id_coded, FIRST_VALUE(department_id) OVER (PARTITION BY pat_enc_csn_id_coded ORDER BY effective_time_jittered_utc) department_id
FROM `shc_core.adt`)

SELECT DISTINCT cens.*,  dep.department_name,
FROM `mining-clinical-decisions.conor_db.abx_cohort_not_censored_no_kids` cens
INNER JOIN adt_dep 
USING (pat_enc_csn_id_coded)
INNER JOIN `shc_core.dep_map` dep
USING (department_id)